In [45]:
class Para:

   index_code = 'CS' # eg. '000300.XSHG' 'CS'
   start_date = '2017-01-01' # eg. '2018-01-01'
   end_date = '2022-01-11'
   interval = 20
   yield_rate_interval = 20

   write_folder_path = 'data/'+str(index_code).lower()+'_space_'+str(interval)+'d_rate_'+str(yield_rate_interval)+'d_'+str(start_date)[0:4]

   # 估值
   valuation_factor = [
      'ep_ratio_ttm',
      'pb_ratio_ttm',
      'sp_ratio_ttm']

   # 成长
   growth_factor = [
      'operating_revenue_growth_ratio_ttm',
      'net_profit_growth_ratio_ttm']

   # 财务质量
   financial_factor = [
      'gross_profit_margin_ttm',
      'total_asset_turnover_ttm']

   # 杠杆
   leverage_factor = [
      'cash_ratio_lf',
      'current_ratio_lf',]

   # 技术
   technology_factor = [
      'MACD_DIFF',
      'MACD_DEA',
      'MACD_HIST',
      'RSI10',
      'SY',
      'BIAS20']

   # 其他
   other_factor = [
      'VOL30',# 换手率
      'VOL60',
      'VOL120',
      'VOLT20',# 波动率
      'VOLT60',
      'AR',# 人气意愿指标
      'BR']

   factor_list = valuation_factor + growth_factor + financial_factor + leverage_factor + technology_factor + other_factor

para = Para()
print(para.write_folder_path)

data/cs_space_20d_rate_20d_2017


In [46]:
import rqdatac

rqdatac.init()

c:\users\millet\miniconda3\envs\torch1.8-gpu\lib\site-packages\rqdatac\client.py:197: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)


In [47]:
from rqdatac import get_trading_dates

trading_dates_list = get_trading_dates(start_date=para.start_date, end_date=para.end_date)
trading_dates_list_interval = trading_dates_list[0: len(trading_dates_list) - para.yield_rate_interval: para.interval]
trading_dates_list_interval

[datetime.date(2017, 1, 3),
 datetime.date(2017, 2, 7),
 datetime.date(2017, 3, 7),
 datetime.date(2017, 4, 6),
 datetime.date(2017, 5, 5),
 datetime.date(2017, 6, 6),
 datetime.date(2017, 7, 4),
 datetime.date(2017, 8, 1),
 datetime.date(2017, 8, 29),
 datetime.date(2017, 9, 26),
 datetime.date(2017, 10, 31),
 datetime.date(2017, 11, 28),
 datetime.date(2017, 12, 26),
 datetime.date(2018, 1, 24),
 datetime.date(2018, 2, 28),
 datetime.date(2018, 3, 28),
 datetime.date(2018, 4, 27),
 datetime.date(2018, 5, 29),
 datetime.date(2018, 6, 27),
 datetime.date(2018, 7, 25),
 datetime.date(2018, 8, 22),
 datetime.date(2018, 9, 19),
 datetime.date(2018, 10, 25),
 datetime.date(2018, 11, 22),
 datetime.date(2018, 12, 20),
 datetime.date(2019, 1, 21),
 datetime.date(2019, 2, 25),
 datetime.date(2019, 3, 25),
 datetime.date(2019, 4, 23),
 datetime.date(2019, 5, 24),
 datetime.date(2019, 6, 24),
 datetime.date(2019, 7, 22),
 datetime.date(2019, 8, 19),
 datetime.date(2019, 9, 17),
 datetime.date(2

In [48]:
from my_utils.data_process import remove_st
from rqdatac import index_components, get_factor, get_price, is_suspended, get_next_trading_date, instruments, all_instruments
import time
import datetime
import numpy as np
import pandas as pd

time_begin = time.time()
for j in range(len(trading_dates_list_interval)):

    print()
    print("number of trading_dates_list_interval: {}/{}".format(j, len(trading_dates_list_interval)))
    print("use time: {}".format(time.time()-time_begin))

    date = trading_dates_list_interval[j]

    # 全部股票
    if para.index_code == 'CS':
        stocks_list = all_instruments(type=para.index_code, date=date)['order_book_id'].values.tolist()
    # hs300
    elif para.index_code == '000300.XSHG':
        stocks_list = index_components(order_book_id=para.index_code, date=date)

    # 剔除 ST 股票
    stocks_list_no_st = remove_st(stocks_list=stocks_list, date=date)
    # 获取因子
    df = get_factor(order_book_ids=stocks_list_no_st, factor=para.factor_list, date=date)
    # 处理多级索引
    df = df.reset_index(level='date')
    # 插入 month 列
    df.insert(loc=0, column='month', value=j)
    # 插入收益率列
    df.insert(loc=0, column='yield_rate', value=np.nan)
    # 插入行业类别列
    df.insert(loc=0, column='sector_code', value=np.nan)
    # 插入板块类别列
    df.insert(loc=0, column='board_type', value=np.nan)

    # 遍历当前截面的每只股票
    for i in df.index.values.tolist():

        # 获取行业和板块信息
        df.loc[i, 'sector_code'] = instruments(order_book_ids=i).sector_code
        df.loc[i, 'board_type'] = instruments(order_book_ids=i).board_type

        # 获取当前日期和未来日期
        day = trading_dates_list_interval[df.loc[i, 'month']]
        day_interval = get_next_trading_date(date=day, n=para.yield_rate_interval)

        # 获取当前日期和未来日期停牌状态
        day_is_suspended = bool(is_suspended(order_book_ids=i, start_date=day, end_date=day).values[0][0])
        day_interval_is_suspended = bool(is_suspended(order_book_ids=i, start_date=day, end_date=day).values[0][0])

        # 判断是否停牌
        if day_is_suspended == False and day_interval_is_suspended == False:

            # 获取当前日期和未来日期的价格
            day_price = get_price(order_book_ids=i, start_date=day, end_date=day, fields='close')
            day_interval_price = get_price(order_book_ids=i, start_date=day_interval, end_date=day_interval, fields='close')

            # 判断是否数据缺失
            if day_price is None or day_interval_price is None:
                print("day_price or day_interval_price is None: {}".format(i))
            else:
                # 判断距离上市天数
                listed_date = datetime.datetime.strptime(instruments(order_book_ids=i).listed_date, '%Y-%m-%d').date()
                how_long_listed = (trading_dates_list_interval[j]-listed_date).days

                if how_long_listed > 90:
                    yield_rate = day_interval_price['close'].item() / day_price['close'].item() - 1
                    df.loc[i, 'yield_rate'] = yield_rate
                else:
                    print('how_long_listed <= 90d: {}'.format(i))

        else:
            print("day_is_suspended or day_interval_is_suspended == True: {}".format(i))

    # 数据预处理

    # Replace infinite updated data with nan
    df.replace(to_replace=[np.inf, -np.inf], value=np.nan, inplace=True)

    # 剔除含有 na 的行
    df_dropna = df.dropna(axis=0, how='any')

    # 选择特征列和标签列
    df_dropna_y = df_dropna[['board_type', 'sector_code', 'yield_rate', 'month', 'date']]
    df_dropna_features = df_dropna.loc[:, 'ep_ratio_ttm':'BR']

    # 中位数去极值
    df_dropna_features = df_dropna_features.clip(
        lower=df_dropna_features.median(axis=0) - 5 * df_dropna_features.mad(axis=0),
        upper=df_dropna_features.median(axis=0) + 5 * df_dropna_features.mad(axis=0),
        axis=1)

    # 标准化
    df_dropna_features = (df_dropna_features - df_dropna_features.mean()) / df_dropna_features.std(ddof=0)

    # 拼接标签列和特征列
    df_dropna_standard = pd.concat([df_dropna_y, df_dropna_features], axis=1)

    # 写入文件
    df_dropna_standard.to_csv(para.write_folder_path + '/' + str(j) + '.csv')

print()
print('Done!')


number of trading_dates_list_interval: 0/61
use time: 0.0019931793212890625
how_long_listed <= 60d: 603819.XSHG
how_long_listed <= 60d: 300572.XSHE
day_is_suspended or day_interval_is_suspended == True: 002462.XSHE
how_long_listed <= 60d: 002822.XSHE
day_is_suspended or day_interval_is_suspended == True: 300449.XSHE
day_is_suspended or day_interval_is_suspended == True: 002745.XSHE
day_is_suspended or day_interval_is_suspended == True: 300212.XSHE
how_long_listed <= 60d: 300565.XSHE
day_is_suspended or day_interval_is_suspended == True: 600551.XSHG
how_long_listed <= 60d: 300573.XSHE
day_is_suspended or day_interval_is_suspended == True: 002159.XSHE
day_is_suspended or day_interval_is_suspended == True: 600655.XSHG


KeyboardInterrupt: 